## Installing PyTorch and Pillow == 0.4

In [1]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

0.4.0
True


In [2]:
!pip install Pillow==4.0.0 

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


## Imports
All the imports are defined here


In [0]:
%matplotlib inline
import os
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random
from PIL import Image
import torch
from torch.autograd import Variable
import PIL.ImageOps    
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!ls '/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset'

final_backup_saved_model.pt  modified_data_loader_saved_model.pt  testing
forged			     saved_model.pt			  training


In [0]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn1 = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(1, 128, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(128, 64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(64, 32, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),

            nn.ReflectionPad2d(1),
            nn.Conv2d(32, 16, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(16),
            
            nn.ReflectionPad2d(1),
            nn.Conv2d(16, 8, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(8),


        )

        self.fc1 = nn.Sequential(
            nn.Linear(8*100*100, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 500),
            nn.ReLU(inplace=True),

            nn.Linear(500, 5))

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [0]:
net = SiameseNetwork().cuda()

In [0]:
# torch.save(net,"/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/saved_model.pt")
# net = torch.load("/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/saved_model.pt")

# filepath = "/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/final_backup_saved_model.pt" # 0.76 threshold

filepath = "/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/modified_data_loader_saved_model.pt"

# torch.save(net.state_dict(), filepath)

#Later to restore:
net.load_state_dict(torch.load(filepath))

In [39]:
!ls '/content/drive/My Drive/Colab Notebooks/axis_data/processed_dataset/'

final_backup_saved_model.pt  modified_data_loader_saved_model.pt  testing
forged			     saved_model.pt			  training


In [0]:
def image_loader(loader, image_name):
    image = Image.open(image_name)
    image = image.convert("L")
    image = loader(image).float()
    image = torch.tensor(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image

data_transforms = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
])

In [0]:
import pandas as pd
import numpy as np
import glob
import os

In [0]:
col_names =  ['filename', 'confidence_score', 'decision']
submision_csv  = pd.DataFrame(columns = col_names)

In [0]:
names = [os.path.basename(x) for x in glob.glob('/content/drive/My Drive/Colab Notebooks/axis_data/final_sub/final_submission_test/*.jpg')]
submision_csv['filename'] = names

In [0]:
TEST_DIR = '/content/drive/My Drive/Colab Notebooks/axis_data/final_sub/final_submission_test/'
USER_REG_DIR = '/content/drive/My Drive/Colab Notebooks/axis_data/final_sub/final_submission_ordered/'

for row in submision_csv.itertuples():
    file = row.filename
    user_id = file[3:][:-8]
    confidence = 0
    user_signs = glob.glob(USER_REG_DIR+user_id+'/*.jpg')
    test_image = TEST_DIR + file
    test_image = image_loader(data_transforms, test_image)
    
    for genuine_sign in user_signs:
      user_sign = image_loader(data_transforms, genuine_sign)
      output1,output2 = net(Variable(user_sign).cuda(),Variable(test_image).cuda())
      euclidean_distance = F.pairwise_distance(output1, output2)
      euclidean_distance = euclidean_distance.cpu().detach().numpy()
      confidence = confidence + euclidean_distance
    
    confidence = confidence/3
    submision_csv.loc[row.Index, 'confidence_score'] = confidence

,filename,confidence_score,decision
0,U11001_380.jpg,[1.1097552],NaN
1,U11001_463.jpg,[1.2613517],NaN
2,U11002_292.jpg,[0.71360356],NaN
3,U11002_340.jpg,[0.68247753],NaN
4,U11002_312.jpg,[1.0208613],NaN


In [0]:
for row in submision_csv.itertuples():
  submision_csv.loc[row.Index, 'decision'] = 0
  score = row.confidence_score
  if score == 0.0:
    continue
  else:
    if score[0] > 0.6:
      submision_csv.loc[row.Index, 'decision'] = 1
    
#   score[0] > 0.6:
    
#     

In [75]:
submision_csv.head()

,filename,confidence_score,decision
0,U11001_380.jpg,[1.1097552],1
1,U11001_463.jpg,[1.2613517],1
2,U11002_292.jpg,[0.71360356],1
3,U11002_340.jpg,[0.68247753],1
4,U11002_312.jpg,[1.0208613],1


In [0]:
# submision_csv.to_csv('/content/drive/My Drive/Colab Notebooks/axis_data/confidence_metric.csv')  
submision_csv.to_csv('/content/drive/My Drive/Colab Notebooks/axis_data/final_sub.csv')